In [70]:
import angr, claripy, archinfo

In [71]:
# use a branch instruction to make the basic block end after the sra
#     beq x12, x0, 12 ; sra x27, x21, x4
byte_code = 0x00060663_41b5d3b3.to_bytes(8, "little")

In [72]:
arch = archinfo.ArchRISCV64()

In [73]:
# create angr project and pass our instrucitons
# angr uses vex to lift the machine code
project = angr.load_shellcode(byte_code, arch=arch, start_offset=0, load_address=0)
entry_state = project.factory.entry_state()

In [ ]:
## optional: print vex ir
project.factory.block(project.entry).vex.pp()
# one can see that x27 is cast into a 8bit value and is then used to rshift x11
# which is wrong: only the lowest 6 bits of x27 may be used for the shift

IRSB {
   t0:Ity_I64 t1:Ity_I8 t2:Ity_I64 t3:Ity_I64 t4:Ity_I1 t5:Ity_I64 t6:Ity_I64 t7:Ity_I64

   00 | ------ IMark(0x0, 4, 0) ------
   01 | t2 = GET:I64(x27)
   02 | t1 = 64to8(t2)
   03 | t3 = GET:I64(x11)
   04 | t0 = Sar64(t3,t1)
   05 | PUT(x7) = t0
   06 | PUT(pc) = 0x0000000000000004
   07 | ------ IMark(0x4, 4, 0) ------
   08 | t5 = GET:I64(x0)
   09 | t6 = GET:I64(x12)
   10 | t4 = CmpEQ64(t6,t5)
   11 | if (t4) { PUT(pc) = 0x10; Ijk_Boring }
   NEXT: PUT(pc) = 0x0000000000000008; Ijk_Boring
}


In [75]:
simulation = project.factory.simulation_manager(entry_state)
simulation = simulation.step()

WARNING  | 2025-09-05 15:30:50,918 | angr.storage.memory_mixins.default_filler_mixin | Filling register x27 with 8 unconstrained bytes referenced from 0x0 (offset 0x0 in main binary (0x0))
WARNING  | 2025-09-05 15:30:50,919 | angr.storage.memory_mixins.default_filler_mixin | Filling register x11 with 8 unconstrained bytes referenced from 0x0 (offset 0x0 in main binary (0x0))
WARNING  | 2025-09-05 15:30:50,922 | angr.storage.memory_mixins.default_filler_mixin | Filling register x0 with 8 unconstrained bytes referenced from 0x4 (offset 0x4 in main binary (0x4))
WARNING  | 2025-09-05 15:30:50,923 | angr.storage.memory_mixins.default_filler_mixin | Filling register x12 with 8 unconstrained bytes referenced from 0x4 (offset 0x4 in main binary (0x4))


In [78]:
# print register x7 as z3 formula
claripy.backends.z3.convert(getattr(simulation.active[0].regs, "x7"))

reg_x11_37_64 >> Concat(0, Extract(7, 0, reg_x27_36_64))